In [231]:
'''
Prediction 1:
scenario_1: The word test appears in at least in one field
scenario_2: All fields are at most the same
scenario_3: One word field
'''
import pandas as pd
import nltk
from nltk.corpus import words
import random
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     /Users/darrylbalderas/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [232]:
# import spam collection messages
names = ['label', 'title']
message_data = pd.read_table("SMSSpamCollection", names=names)

In [233]:
# Remove rows that contain spam label
message_data = message_data.drop(message_data[message_data.label == "spam"].index)

In [234]:
# Drop label column
message_data = message_data.drop('label', axis=1)

In [235]:
message_data = message_data.reset_index(drop=True)
message_data.head(10)

,title
0,"Go until jurong point, crazy.. Available only ..."
1,Ok lar... Joking wif u oni...
2,U dun say so early hor... U c already then say...
3,"Nah I don't think he goes to usf, he lives aro..."
4,Even my brother is not like to speak with me. ...
5,As per your request 'Melle Melle (Oru Minnamin...
6,I'm gonna be home soon and i don't want to tal...
7,I've been searching for the right words to tha...
8,I HAVE A DATE ON SUNDAY WITH WILL!!
9,Oh k...i'm watching here:)


In [236]:
english_words = dict.fromkeys(words.words(), None)

def is_word(word):
    try:
        x = english_words[word]
        return True
    except:
        return False


def get_false_word_indexes(data, percentage):
    indexes = []
    for index, message in enumerate(data['title']):
        false_word_count = 0
        sentence = message.split()
        for word in sentence:
            if not is_word(word):
                false_word_count += 1
                if false_word_count/len(message) > false_word_percentage:
                    indexes.append(index)
                    break
    return indexes

def remove_single_words(data):
    indexes = []
    for index, message in enumerate(data['title']):
        sentence = message.split()
        if len(sentence) == 1:
            indexes.append(index)
    return indexes
    

In [237]:
%%time
false_word_percentage = 0.02
indexes = get_false_word_indexes(message_data, false_word_percentage)
print('{} number of indexes will be deleted'.format(len(indexes)))

4741 number of indexes will be deleted
CPU times: user 15.9 ms, sys: 679 µs, total: 16.6 ms
Wall time: 16.1 ms


In [238]:
original_size = message_data.shape[0]
real_messages = message_data.drop(message_data.index[indexes]).reset_index(drop=True)
print('Data size went from {} to {}'.format(original_size, real_messages.shape[0]))

Data size went from 4825 to 84


In [239]:
real_messages.head()

,title
0,I see the letter B on my car
1,Ok i am on the way to home hi hi
2,I see a cup of coffee animation
3,Keep yourself safe for me because I need you a...
4,Or ill be a little closer like at the bus stop...


In [240]:
%%time
indexes = remove_single_words(real_messages)
print('{} number of indexes will be deleted'.format(len(indexes)))
no_single_messages = real_messages.drop(real_messages.index[indexes]).reset_index(drop=True)
print('Data size went from {} to {}'.format(real_messages.shape[0], single_messages.shape[0]))

8 number of indexes will be deleted
Data size went from 84 to 76
CPU times: user 1.39 ms, sys: 93 µs, total: 1.48 ms
Wall time: 1.41 ms


In [241]:
def add_test_message(sentence):
    words = sentence.split()
    choice = random.randint(0,len(words) + 1)
    update_sentence = ""
    for index, word in enumerate(words):
        if index == choice:
            update_sentence += "test "
        if index == len(words) - 1:
            update_sentence += word
        else:
            update_sentence += (word + " ")
    if choice == len(words):
        update_sentence += " test"
    return update_sentence


In [252]:
scenario_one = pd.DataFrame(no_single_messages)
scenario_one['subtitle'] = scenario_one['title']
scenario_one['message'] = scenario_one['title']
for i in range(30):
    scenario_one['subtitle'] = scenario_one['subtitle'].sample(frac=1).reset_index(drop=True)
    scenario_one['message'] = scenario_one['message'].sample(frac=1).reset_index(drop=True)
    scenario_one['title'] = scenario_one['subtitle'].sample(frac=1).reset_index(drop=True)

In [253]:
for index in scenario_one.index:
    columns = list(scenario_one.columns)
    prob = random.random()
    if prob < 0.33:
        choice = random.choice(columns)
        scenario_one.loc[index][choice] = add_test_message(scenario_one.loc[index][choice])
    elif prob < 0.66:
        choice = random.choice(columns)
        scenario_one.loc[index][choice] = add_test_message(scenario_one.loc[index][choice])
        columns.remove(choice)
        choice = random.choice(columns)
        scenario_one.loc[index][choice] = add_test_message(scenario_one.loc[index][choice])
    else:
        for column in columns:
            scenario_one.loc[index][column] = add_test_message(scenario_one.loc[index][column])

In [255]:
scenario_one.head()

,title,subtitle,message
0,test I can take you at like noon,"Well, I meant as opposed to my drunken night o...",I could ask carlos if we could get more if any...
1,I want to tell you how bad I feel test that ba...,Carlos is taking his sweet time as usual so le...,Ok test anyway no need to change with what you...
2,I dont want to hear anything,I cant keep talking to people if test am not s...,I dont want to hear anything test
3,I dont have i shall test buy one dear,I think i am disturbing her da,I cant keep talking to people if am not sure i...
4,Ok ill tell the company,Jesus armand really is trying to tell everybod...,I want to be there so i can kiss you and test ...
